# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [1]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [2]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 14.24093 (60.79966)
Val Loss (Med): 14.24093 (60.79966)
Median Size (Max): 3 (18)
Median complexity (Max): 9 (351)
Time (s): 0.12213

Generation 2/10 [///////////                                       ]
Train Loss (Med): 13.86337 (20.54475)
Val Loss (Med): 13.86337 (20.54475)
Median Size (Max): 3 (20)
Median complexity (Max): 9 (651)
Time (s): 0.20873

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 13.58168 (17.94969)
Val Loss (Med): 13.58168 (17.94969)
Median Size (Max): 3 (18)
Median complexity (Max): 9 (216)
Time (s): 0.30500

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 13.58167 (17.94969)
Val Loss (Med): 13.58167 (17.94969)
Median Size (Max): 3 (18)
Median complexity (Max): 9 (478)
Time (s): 0.44846

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 13.58167 (17.94969)
Val Loss 

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loaded population from /tmp/tmp91lebnyr/population.json of size = 200
Completed 100% [====================]
saving final population as archive...
score: 0.8893989188100573


You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `error` and `complexity`, and the second run is optimizing `average_precision_score` and `size`.

In [4]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    scorer="log",
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 0.54930 (0.69315)
Val Loss (Med): 0.54930 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (270)
Time (s): 0.05091

Generation 2/10 [///////////                                       ]
Train Loss (Med): 0.54848 (0.69315)
Val Loss (Med): 0.54848 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (54)
Time (s): 0.08855

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 0.54847 (0.69315)
Val Loss (Med): 0.54847 (0.69315)
Median Size (Max): 5 (10)
Median complexity (Max): 6 (54)
Time (s): 0.12188

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 0.54847 (0.69315)
Val Loss (Med): 0.54847 (0.69315)
Median Size (Max): 1 (10)
Median complexity (Max): 2 (52)
Time (s): 0.15164

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 0.54846 (0.69315)
Val Loss (Med): 0.54846 (0.693

In [5]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    #load_population=pop_file,
    objectives=["error", "complexity"],
    scorer="average_precision_score",
    max_gens=1,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/1 [//////////////////////////////////////////////////]
Train Loss (Med): 0.46115 (0.31675)
Val Loss (Med): 0.46115 (0.31675)
Median Size (Max): 5 (9)
Median complexity (Max): 6 (120)
Time (s): 0.04622

saving final population as archive...
Logistic(Sum(-1.2485952,Add(-1.25,0.08*AIDS)))
score: 0.54
